<a href="https://colab.research.google.com/github/RC11-SkillsClass2022-23/22136945/blob/main/Ceel_assignment/ApplyingKnowledge/FeatureHomeworkVideoClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! git clone https://github.com/kenshohara/video-classification-3d-cnn-pytorch.git

fatal: destination path 'video-classification-3d-cnn-pytorch' already exists and is not an empty directory.


In [ ]:
%cd video-classification-3d-cnn-pytorch/

/content/video-classification-3d-cnn-pytorch


In [ ]:
import tensorflow as tf
import PIL
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dropout, Dense, Softmax)
from tensorflow.keras.applications import mobilenet as _mobilenet
import random 
import json
import os
import moviepy
from moviepy.editor import *
from moviepy import *
import shutil
import numpy as np

In [ ]:
def loadVideosToInfile(folder):
    videos = os.listdir(folder)
    with open('input', 'w') as inputfile:
      for v in videos:
        if v.split('.')[-1] == 'mp4':
          inputfile.write(v+'\n')

In [ ]:
def readAllOutput(outputFolder):
    # create an empty dictionary called 'dictionaries' to which you add the dictionary entries read 
    # from the output files
    dictionaries = {}
    # Read all output file names to a variable output_files (a list that contains all file names)
    output_files = os.listdir(outputFolder)
    # Start of the for loop to loop through all files, read them, and add their contents to 'dictionaries'
    for f in output_files:
        # first test if the file extension is a '.json file'
            if f.split('.')[-1] == 'json':
                file = open(os.path.join(outputFolder, f))
                segments = json.load(file)
            # if it is a json file, we open the file
            
            # Then load the file content to a variable called 'segments' with the json library

            # another for loop, to add the content of segments to dictionaries. As a key use the moviename.
                for s in segments:
                    dictionaries[s['video']] = s
    
    return dictionaries

In [ ]:
def searchForMatch(keyFilmName, featureDictionary, frame = None, fps = 25, nBestMatches = 1):
    # create variables to keep track of the best clips

    fragments = []
    
    # first read the features of the keyFilm into keyFeatures
    keyClips = featureDictionary[keyFilmName]['clips']
    keyFeatures = []
    for c in keyClips:
        keyFeatures.append(np.array(c['features']))
        
    if frame:
        ft = 16/fps
        
        sf = int(frame[0]/ft)
        ef = int(min(frame[1]/ft, len(keyFeatures)-1))
        
        keyFeatures = keyFeatures[sf:ef]
    
    # now enter a for loop to go through all films, and see what the best matching sequence is 
    # (we make sure not to check the keyFilm itself by checking it's not that filename)
    for film in featureDictionary.keys():
        if not film == keyFilmName:
            filmClips = featureDictionary[film]['clips']
            filmFeatures = []
            for c in filmClips:
                filmFeatures.append(np.array(c['features']))
            for i in range(len(filmFeatures)-len(keyFeatures)):
                distance = 0
                for j in range(len(keyFeatures)):
                    d = np.linalg.norm(filmFeatures[i+j]-keyFeatures[j])
                    distance += d
            
                distance = distance/len(keyFeatures)
                    
                a = fragmentsExists(fragments, [film, i, len(keyFeatures), distance])
                
                if a:
                    fragments = a
                elif len(fragments) < nBestMatches:
                    fragments.append([film, i, len(keyFeatures), distance])
                    fragments.sort(key = lambda x : x[-1])
                elif distance < fragments[-1][-1]:
                    fragments.pop()
                    fragments.append([film, i, len(keyFeatures), distance])
                    fragments.sort(key = lambda x : x[-1])

              

            # so now here 'film' is a filename (which is the key in the feature dictionary of that film)
            # first load the features in a 'filmFeatures' list like we did for key Features
            # then look at the provided code from the lecture to see how to iterate through the other
            # list of features to find the best segment. Now just make sure to also register what film
            # name the best matching segment is from. 
            
    # create a list containing the filename, start and nr of clips
    
    return fragments

In [ ]:
def fragmentsExists(fragments, nf):
    for f in fragments:
        r = range(nf[1], nf[1]+nf[2])
        if f[0] == nf[0] and (f[1] in r or f[1]+f[2] in r):
            if f[-1] > nf[-1]:
                fragments.remove(f)
                fragments.append(nf)
                fragments.sort(key = lambda x : x[-1])
            return fragments
    return False

In [ ]:
def extractFragment(fragment, filmFolder):
    # here adapt the code from the lecture, but make it so that it works for any input fragment list
    # the output of the function above: [bestRangeFilmName, bestRangeStart, len(keyFeatures)]

    filmFileName = fragment[0]
    start = fragment[1]
    nrClips = fragment[2]
    
    film = VideoFileClip(os.path.join(filmFolder, filmFileName))
    tsfps = film.fps
    tsFrameLength = 1/tsfps
    
    nrFrames = nrClips * 16
    startFrame = start * 16
    
    startTime = startFrame * tsFrameLength
    endTime = startTime + nrFrames * tsFrameLength
    
    filmFragment = film.subclip(startTime, endTime)
    
    
    return filmFragment

In [ ]:
loadVideosToInfile('/content/drive/MyDrive/SkillWorkshop/VideoFeatureTHINK')

In [ ]:
# run the model to process all films currently in the input_films folder, and save it to a json file, here named by the date

!python main.py --input input --video_root /content/drive/MyDrive/SkillWorkshop/VideoFeatureTHINK --output /content/drive/MyDrive/SkillWorkshop/VideoOutFeature/output1210-think.json --model /content/drive/MyDrive/SkillWorkshop/resnet-34-kinetics-cpu.pth --mode feature --no_cuda


loading model /content/drive/MyDrive/SkillWorkshop/resnet-34-kinetics-cpu.pth
/content/drive/MyDrive/SkillWorkshop/VideoFeatureTHINK/THINK.mp4
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-li

In [ ]:
input_films_processed = os.listdir('/content/drive/MyDrive/SkillWorkshop/ScrapedVideo')
for file in input_films_processed:
    try:
        shutil.move(os.path.join('/content/drive/MyDrive/SkillWorkshop/ScrapedVideo',file),'/content/drive/MyDrive/SkillWorkshop/VideoProcessedFilms')
    except:
        print(file + ' not moved')

.ipynb_checkpoints not moved


In [ ]:
dicts = readAllOutput('/content/drive/MyDrive/SkillWorkshop/VideoOutFeature')

In [ ]:
keyName = 'THINK.mp4'
s = 0
d = 6
fragment = searchForMatch(keyName, dicts, frame = [s, s+d], nBestMatches = 30)

In [ ]:
fragment

[['frame-099599.mp4', 21, 5, 2.951256350375716],
 ['Cobbing-Bob_Fencott-PC_Performing-Concrete-Poetry_1985.mp4',
  2229,
  5,
  11.378539507393262],
 ['Cobbing-Bob_Fencott-PC_Performing-Concrete-Poetry_1985.mp4',
  2917,
  5,
  11.712974842717882],
 ['Cobbing-Bob_Fencott-PC_Performing-Concrete-Poetry_1985.mp4',
  2767,
  5,
  11.884339021906468],
 ['Cobbing-Bob_Fencott-PC_Performing-Concrete-Poetry_1985.mp4',
  2214,
  5,
  11.898587626566883],
 ['Cobbing-Bob_Fencott-PC_Performing-Concrete-Poetry_1985.mp4',
  2253,
  5,
  11.943662230043913],
 ['ZeddNickElectraElfTheBeginning2005.mp4', 938, 5, 12.260919630367763],
 ['frame-099599.mp4', 0, 5, 12.262133316088901],
 ['Cobbing-Bob_Fencott-PC_Performing-Concrete-Poetry_1985.mp4',
  2565,
  5,
  12.291376568982775],
 ['Cobbing-Bob_Fencott-PC_Performing-Concrete-Poetry_1985.mp4',
  2558,
  5,
  12.317132008620238],
 ['ZeddNickElectraElfTheBeginning2005.mp4', 966, 5, 12.338463189962102],
 ['Cobbing-Bob_Fencott-PC_Performing-Concrete-Poetry_198

In [ ]:
fragmentFilm = extractFragment(fragment[24], '/content/drive/MyDrive/SkillWorkshop/VideoProcessedFilms')

In [ ]:
fragmentFilm.ipython_display(maxduration = 500)

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [ ]:
# save the film fragment

fragmentFilm.write_videofile("/content/drive/MyDrive/SkillWorkshop/OUTFINAL/think/think1.mp4", audio_codec='aac')

Moviepy - Building video /content/drive/MyDrive/SkillWorkshop/OUTFINAL/street/street4.mp4.
MoviePy - Writing audio in street4TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/SkillWorkshop/OUTFINAL/street/street4.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/SkillWorkshop/OUTFINAL/street/street4.mp4
